<p style="font-family: Arial; font-size:2.75em;color:blue; font-style:bold"><br>
Projet 6 :<br><br>
Catégorisez automatiquement des questions<br><br>
</p><br>

* **Data Source / Data Source :** 
    - Data :
        - Outils d'extraction de data : https://data.stackexchange.com/stackoverflow
        - Script : https://github.com/EricJacquesPro/TextCategorization/blob/master/SQL/ExtractionDonnees.sql
    - Source Python : https://github.com/EricJacquesPro/TextCategorization
* **Description / Description : ** Projet 6 de la formation Data Scientist Proposée par OpenClassroom
    - Analyse
    - Entrainement
* **Auteur / Author : ** Eric JACQUES 
* **Date : ** 06/08/2019 

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
I - Sélection des datas entrantes</p>

Stack overfloaw met a disposition un outils en ligne permettant d'extraire le contenu de post diffusés sur le site internet.
(https://data.stackexchange.com/stackoverflow)<br>
En se basant sur la structure de la base de données(https://i.stack.imgur.com/JZHsN.png), il en ressort des données (tables) plus intéressantes que d'autres : Post, type de post, Tag, tags des posts <br>
Pour notre cas, tous les posts ne sont pas intéressants : Supprimé, score null, sans réponse, type de post....<br>
<br>
Parmis les posts, les données les plus explicites qui permettent de connaitre leurs sujets sont le body.<br>
<br>
Script téléchargeable sur https://github.com/EricJacquesPro/TextCategorization/blob/master/SQL/ExtractionDonnees.sql



<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
II - Data cleaning & preprocessing</p>

In [1]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.insert(0, './Python/')

In [2]:
'''
python class for the data engineering (reading, cleaning, training...)
localised in ./Python
'''
from tagText import TagText 

In [3]:
tagText = TagText()
print(tagText.urlDirectory)#folder for cvs file generated by SQL query (cf. I) 
print(tagText.fileName)#name of the cvs file generated by SQL query (cf. I)
data_question = tagText.read_source()
data_question.head()

Data/
QuestionVsTags.csv


,body,title,tags,SelectedTags,Name
0,<p>Here is a piece of C++ code that shows some...,Why is processing a sorted array faster than p...,<java><c++><performance><optimization><branch-...,"java,c++,performance,optimization,",Question
1,<p>I accidentally committed the wrong files to...,How do I undo the most recent local commits in...,<git><version-control><git-commit><undo>,"git,version-control,git-commit,",Question
2,<p>I want to delete a branch both locally and ...,How do I delete a Git branch locally and remot...,<git><git-branch><git-remote>,"git,git-branch,",Question
3,<blockquote>\n <p><strong>Moderator Note:</st...,What is the difference between 'git pull' and ...,<git><git-pull><git-fetch>,"git,",Question
4,"<p>I've been messing around with <a href=""http...",What is the correct JSON content type?,<json><http-headers><content-type>,"json,http-headers,content-type,",Question


In [4]:
data_question.describe().T

,count,unique,top,freq
body,4000,4000,"<p>I've tried checking <a href=""https://stacko...",1
title,4000,4000,Deleting DataFrame row in Pandas based on colu...,1
tags,4000,3413,<git>,64
SelectedTags,4000,3054,"git,",139
Name,4000,1,Question,4000


In [5]:
print(tagText.stopwords)
'''Stop word issu to nltk.corpus'''

{"that'll", 'd', 'once', 'hadn', "shan't", 'their', 'her', 'above', 'whom', 'for', 'up', 'doesn', 'ourselves', 'a', "weren't", 'each', 'mightn', 'isn', 'theirs', 'after', 'myself', 'had', 'how', 'too', "hadn't", 'himself', 'wasn', 'over', 'very', 'if', 'have', 'can', 'such', 'themselves', 'out', "should've", 'has', 'yourselves', 'but', 'now', "wouldn't", 'didn', 'when', "you'd", 'weren', 'm', 'ma', "wasn't", 'them', "mightn't", 'are', 'to', 'between', 'most', 'so', 'here', "isn't", 'y', 'against', "you've", 'wouldn', 'about', 'under', 'some', 'through', 'why', 'until', 'all', 'doing', 'ours', "shouldn't", 'while', 'this', 'he', 'you', 'or', 'just', 'yourself', 'only', 'that', "doesn't", "mustn't", 'where', 'my', 'more', 'should', 've', 'as', 'did', 'before', "couldn't", 'mustn', 'other', "it's", 'it', "won't", 'by', 'is', 'do', 'from', 'was', 'any', 'don', 'him', 'of', 's', 'its', 'aren', 'what', 'further', 'she', 'who', 'hasn', 'these', 'o', 'own', 'i', 'because', 'will', 'herself', '

'Stop word issu to nltk.corpus'

In [6]:
data_question['body'][1:9]

1    <p>I accidentally committed the wrong files to...
2    <p>I want to delete a branch both locally and ...
3    <blockquote>\n  <p><strong>Moderator Note:</st...
4    <p>I've been messing around with <a href="http...
5    <p>What is the use of the <code>yield</code> k...
6    <p>After reading <a href="http://groups.google...
7    <p>I mistakenly added files to git using the c...
8    <p>I don't want to rename a remote branch, as ...
Name: body, dtype: object

nltk stop word are not efficient.
There are lot of html and punctuation caracters

In [7]:
data_preprocessed = data_question['body'].apply(tagText.preprocessing)
print(data_preprocessed[1:9])

1    accidentally committed wrong files git pushed ...
2    want delete branch locally remotely failed att...
3    moderator note given question already sixtysev...
4    ive messing around json time pushing text hurt...
5    use yield keyword python do example im trying ...
6    reading hidden features dark corners cstl comp...
7    mistakenly added files git using command git a...
8    want rename remote branch described rename mas...
Name: body, dtype: object


In [8]:
data_question.head()

,body,title,tags,SelectedTags,Name
0,<p>Here is a piece of C++ code that shows some...,Why is processing a sorted array faster than p...,<java><c++><performance><optimization><branch-...,"java,c++,performance,optimization,",Question
1,<p>I accidentally committed the wrong files to...,How do I undo the most recent local commits in...,<git><version-control><git-commit><undo>,"git,version-control,git-commit,",Question
2,<p>I want to delete a branch both locally and ...,How do I delete a Git branch locally and remot...,<git><git-branch><git-remote>,"git,git-branch,",Question
3,<blockquote>\n <p><strong>Moderator Note:</st...,What is the difference between 'git pull' and ...,<git><git-pull><git-fetch>,"git,",Question
4,"<p>I've been messing around with <a href=""http...",What is the correct JSON content type?,<json><http-headers><content-type>,"json,http-headers,content-type,",Question


In [9]:
data_preprocessed.describe()

count                                                  4000
unique                                                 4000
top       anyone explain difference servermappath server...
freq                                                      1
Name: body, dtype: object

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
III - Data - Bag of words</p>

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
III.1 - Data - Bag of words - most frequent data</p>

In [10]:
import nltk
print(nltk.word_tokenize(data_preprocessed[0])[0:10])
print(nltk.tokenize.WhitespaceTokenizer().tokenize(data_preprocessed[0])[0:10])
# Tokenize using Punctuations
print(nltk.tokenize.WordPunctTokenizer().tokenize(data_preprocessed[0])[0:10])
print(nltk.tokenize.TreebankWordTokenizer().tokenize(data_preprocessed[0])[0:10])

['piece', 'c', 'code', 'shows', 'peculiar', 'behavior', 'strange', 'reason', 'sorting', 'data']
['piece', 'c', 'code', 'shows', 'peculiar', 'behavior', 'strange', 'reason', 'sorting', 'data']
['piece', 'c', 'code', 'shows', 'peculiar', 'behavior', 'strange', 'reason', 'sorting', 'data']
['piece', 'c', 'code', 'shows', 'peculiar', 'behavior', 'strange', 'reason', 'sorting', 'data']


<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
III.2 - Data - Bag of words - Nb of occurence</p>

In [11]:
import nltk
from collections import defaultdict
    
corpora = defaultdict(list)
        
tokenizer = nltk.RegexpTokenizer(r'\w+')
for id, row in data_preprocessed.iteritems():
    for word in tokenizer.tokenize(row):
        if not word in corpora :
            corpora[word] = 0
        #print(word)
        corpora[word] += 1
    

#print(corpora)
#print(sorted(corpora.items(), reverse=True,  key=lambda x: x[1]))

In [12]:
import matplotlib.pyplot as plt
minimum_count = 200
def count_word_occurencies(df):
        corpora = defaultdict(list)                
        tokenizer = nltk.RegexpTokenizer(r'\w+')
        for id, row in df.iteritems():
            for word in tokenizer.tokenize(row):
                if not word in corpora :
                    corpora[word] = 0
                #print(word)
                corpora[word] += 1
        corpora = dict((k, v) for k, v in corpora.items() if ((v > minimum_count)))
        sorted(corpora)
        max_occurence = max(corpora.values())
        min_occurence = (corpora.values())
        min_occurence = min(corpora.values())
        quartil1 =  np.percentile(list(corpora.values()), 25)
        quartil3 =  np.percentile(list(corpora.values()), 75)
        inter_quartil_range = quartil3 - quartil1
        lower_bound = quartil1 -(1.5 * inter_quartil_range) 
        upper_bound = quartil3 +(1.5 * inter_quartil_range) 
        print (min_occurence)
        print (max_occurence)
        print (quartil1)
        print (quartil3)
        print (inter_quartil_range)
        print (lower_bound)
        print (upper_bound)
        #plt.boxplot(corpora.items())
        #print(corpora.values())
        #quartil1 = np.percentile(corpora.values(),25)
        corpora = dict((k, v) for k, v in corpora.items() if ((v > 1)))# and (v > lower_bound) and (v < upper_bound)))
        return (sorted(corpora.items(), reverse=True,  key=lambda x: x[1]))

dico = count_word_occurencies(data_preprocessed)


In [13]:
dico = count_word_occurencies(data_preprocessed)

204
1413
245.5
474.25
228.75
-97.625
817.375


In [14]:
len(dico)

128

In [15]:
dico

[('like', 1413),
 ('use', 1244),
 ('using', 1232),
 ('way', 1130),
 ('want', 1123),
 ('im', 1088),
 ('code', 1051),
 ('git', 1035),
 ('file', 1021),
 ('would', 962),
 ('get', 950),
 ('new', 861),
 ('one', 855),
 ('this', 826),
 ('string', 786),
 ('1', 694),
 ('know', 637),
 ('function', 635),
 ('example', 584),
 ('need', 571),
 ('class', 567),
 ('ive', 560),
 ('error', 554),
 ('something', 548),
 ('c', 545),
 ('it', 511),
 ('return', 495),
 ('following', 494),
 ('files', 492),
 ('work', 489),
 ('id', 479),
 ('var', 478),
 ('data', 473),
 ('0', 468),
 ('2', 461),
 ('python', 460),
 ('branch', 459),
 ('set', 458),
 ('public', 456),
 ('method', 439),
 ('value', 434),
 ('list', 427),
 ('object', 426),
 ('javascript', 418),
 ('also', 410),
 ('div', 408),
 ('find', 407),
 ('int', 392),
 ('see', 379),
 ('line', 374),
 ('question', 371),
 ('make', 357),
 ('tried', 354),
 ('command', 352),
 ('array', 351),
 ('time', 349),
 ('create', 345),
 ('name', 345),
 ('used', 344),
 ('could', 343),
 ('3',

In [16]:
print(tagText.count_word_occurencies(data_preprocessed))

[('like', 1413), ('use', 1244), ('using', 1232), ('way', 1130), ('want', 1123), ('im', 1088), ('code', 1051), ('git', 1035), ('file', 1021), ('would', 962), ('get', 950), ('new', 861), ('one', 855), ('this', 826), ('string', 786), ('1', 694), ('know', 637), ('function', 635), ('example', 584), ('need', 571), ('class', 567), ('ive', 560), ('error', 554), ('something', 548), ('c', 545), ('it', 511), ('return', 495), ('following', 494), ('files', 492), ('work', 489), ('id', 479), ('var', 478), ('data', 473), ('0', 468), ('2', 461), ('python', 460), ('branch', 459), ('set', 458), ('public', 456), ('method', 439), ('value', 434), ('list', 427), ('object', 426), ('javascript', 418), ('also', 410), ('div', 408), ('find', 407), ('int', 392), ('see', 379), ('line', 374), ('question', 371), ('make', 357), ('tried', 354), ('command', 352), ('array', 351), ('time', 349), ('name', 345), ('create', 345), ('used', 344), ('3', 343), ('could', 343), ('difference', 334), ('trying', 331), ('possible', 32

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
III.2 - Data - Bag of words - TF-IDF</p>

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

stemmer = nltk.stem.porter.PorterStemmer()
stopword = nltk.corpus.stopwords.words('english')

#X = data_preprocessed.unique()
X = data_preprocessed[0:9].unique()

def stem_and_filter(doc):
    tokens = [stemmer.stem(w) for w in analyzer(doc)]
    return [token for token in tokens if token.isalpha()]

analyzer = TfidfVectorizer().build_analyzer()
cv = TfidfVectorizer(lowercase=True, stop_words="english", analyzer=stem_and_filter, min_df=0.00, max_df=0.3)  # we remove words if it appears in more than 30 % of the corpus (not found stopwords like Box, Christmas and so on)
word_count_vector = cv.fit_transform(X)
print("TF_IDF_matrix :", word_count_vector.shape, "of", word_count_vector.dtype)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["tf_idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['tf_idf_weights'])


TF_IDF_matrix : (9, 256) of float64


,tf_idf_weights
file,2.203973
tri,2.203973
remot,2.203973
branch,2.203973
differ,2.203973
start,2.203973
thi,2.203973
delet,2.203973
piec,2.203973
return,2.203973


In [18]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.decomposition import LatentDirichletAllocation
no_topics = 20
documents = data_preprocessed.unique()

tf_vectorizer = CountVectorizer(
    max_df=0.95,
    min_df=2, 
    max_features=1000,
    stop_words='english'
)
tf = tf_vectorizer.fit_transform(documents)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

'''
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic :", topic_idx, ":" )
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
'''

no_top_words = 5
tagText.display_topics(lda, tf_vectorizer.get_feature_names(), no_top_words)
print(tf_vectorizer.get_feature_names())
print(lda)


Topic : 0 :
list python key values print
Topic : 1 :
string object var variable test
Topic : 2 :
function return var code value
Topic : 3 :
android application file version data
Topic : 4 :
public new method void string
Topic : 5 :
use difference question php know
Topic : 6 :
table studio visual eclipse html
Topic : 7 :
way number want best easy
Topic : 8 :
int code long main lines
Topic : 9 :
file error python install command
Topic : 10 :
im code file script use
Topic : 11 :
language good things programming right
Topic : 12 :
like id im way using
Topic : 13 :
div css height content width
Topic : 14 :
loadmodule cprogram override virtual list
Topic : 15 :
search certain grep contain pattern
Topic : 16 :
git branch commit file files
Topic : 17 :
run image windows command process
Topic : 18 :
class methods java private interface
Topic : 19 :
data user request server database
['000', '10', '100', '1000', '11', '12', '123', '13', '14', '15', '16', '20', '23', '25', '26', '30', '32', '50', 

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
IV - Data - Tag genetaor </p>

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
IV.1 - Data - Tag generator - Unsupervised - By nb of occurence</p>

In [19]:
dict_tag = count_word_occurencies(data_preprocessed)
dict_tag = str(dict_tag)
dict_tag


204
1413
245.5
474.25
228.75
-97.625
817.375


"[('like', 1413), ('use', 1244), ('using', 1232), ('way', 1130), ('want', 1123), ('im', 1088), ('code', 1051), ('git', 1035), ('file', 1021), ('would', 962), ('get', 950), ('new', 861), ('one', 855), ('this', 826), ('string', 786), ('1', 694), ('know', 637), ('function', 635), ('example', 584), ('need', 571), ('class', 567), ('ive', 560), ('error', 554), ('something', 548), ('c', 545), ('it', 511), ('return', 495), ('following', 494), ('files', 492), ('work', 489), ('id', 479), ('var', 478), ('data', 473), ('0', 468), ('2', 461), ('python', 460), ('branch', 459), ('set', 458), ('public', 456), ('method', 439), ('value', 434), ('list', 427), ('object', 426), ('javascript', 418), ('also', 410), ('div', 408), ('find', 407), ('int', 392), ('see', 379), ('line', 374), ('question', 371), ('make', 357), ('tried', 354), ('command', 352), ('array', 351), ('time', 349), ('create', 345), ('name', 345), ('used', 344), ('could', 343), ('3', 343), ('difference', 334), ('trying', 331), ('possible', 3

In [20]:
def unsupervised_tag(dict_word_key, new_question, number_max_tag):
    tags = [word for word in new_question.split() if (word in dict_word_key)]
    return tags[0:number_max_tag]

print(unsupervised_tag(dict_tag, "How save my code source with soft like git", 20))
print(unsupervised_tag(dict_tag, "How save my code source with soft like like git", 2))
#for word in "like git branche totot, test".split() if (word in dict_word_key)

['code', 'with', 'like', 'git']
['code', 'with']


In [21]:
tagText.unsupervised_tag(dict_tag, "How save my code source with soft like git", 2)

['code', 'with']

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
IV.2-a - Data - Tag generator - Unsupervised - LDA</p>

In [22]:
tf = word_count_vector

from sklearn.decomposition import LatentDirichletAllocation
no_topics = 20
'''
tf_vectorizer = COuntVectorizer(max_df)
tf = tf_vectorizer.fit_transform(documents)
'''
# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)


def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic :", topic_idx, ":" )
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 5
display_topics(lda, cv.get_feature_names(), no_top_words)
# reprendre forcement avec countVectorizer

Topic : 0 :
reason applicationxjavascript mani keyword pull
Topic : 1 :
branch primari anomali method stop
Topic : 2 :
anomali wrong start stdendl branch
Topic : 3 :
faster method subsequ browser selfleftchild
Topic : 4 :
moder sixtyseven differ getchildcandid target
Topic : 5 :
remot case show fail sinc
Topic : 6 :
static self uptod staticcastdoubleclock random
Topic : 7 :
anoth modul uptod might complangcmoder
Topic : 8 :
algorithm thing python unsort remotesoriginbugfix
Topic : 9 :
rest server unsort whether api
Topic : 10 :
follow them systemoutprintlnsum describ goe
Topic : 11 :
from seen them exampl strang
Topic : 12 :
int here systemoutprintlnsum commit loop
Topic : 13 :
distanc seen getchildcandidatesself written silli
Topic : 14 :
textxjavascript gather make mistakenli mani
Topic : 15 :
systemnanotim sinc branch case given
Topic : 16 :
again text primari success call
Topic : 17 :
rest method remot renam similar
Topic : 18 :
need support accident rnd around
Topic : 19 :
branch 

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.decomposition import LatentDirichletAllocation
no_topics = 20
documents = data_preprocessed[0:9].unique()

tf_vectorizer = CountVectorizer(
    max_df=0.95,
    min_df=2, 
    max_features=1000,
    stop_words='english'
)
tf = tf_vectorizer.fit_transform(documents)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

'''
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic :", topic_idx, ":" )
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
'''

no_top_words = 5
tagText.display_topics(lda, tf_vectorizer.get_feature_names(), no_top_words)
print(tf_vectorizer.get_feature_names())
print(lda)


Topic : 0 :
branch remote local git pushed
Topic : 1 :
id question start similar returned
Topic : 2 :
pushed git commit local new
Topic : 3 :
start commit pushed branch piece
Topic : 4 :
returned similar want commit git
Topic : 5 :
local git start want branch
Topic : 6 :
undo remote commit deleted include
Topic : 7 :
git branch commit new remote
Topic : 8 :
main git id local branch
Topic : 9 :
question start code similar include
Topic : 10 :
int id local main include
Topic : 11 :
main undo question returned similar
Topic : 12 :
result piece code files returned
Topic : 13 :
pushed code local new branch
Topic : 14 :
int code include start main
Topic : 15 :
pushed question result branch deleted
Topic : 16 :
result pull new include want
Topic : 17 :
similar commit branch deleted question
Topic : 18 :
include want piece int deleted
Topic : 19 :
start new int result pull
['branch', 'code', 'commit', 'deleted', 'files', 'git', 'id', 'include', 'int', 'local', 'main', 'new', 'piece', 'pull', '

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.decomposition import LatentDirichletAllocation
no_topics = 20
documents = data_preprocessed.unique()

tf_vectorizer = CountVectorizer(
    max_df=0.95,
    min_df=2, 
    max_features=1000,
    stop_words='english'
)
tf = tf_vectorizer.fit_transform(documents)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

'''
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic :", topic_idx, ":" )
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
'''

no_top_words = 5
tagText.display_topics(lda, tf_vectorizer.get_feature_names(), no_top_words)
print(tf_vectorizer.get_feature_names())
print(lda)


Topic : 0 :
list python key values print
Topic : 1 :
string object var variable test
Topic : 2 :
function return var code value
Topic : 3 :
android application file version data
Topic : 4 :
public new method void string
Topic : 5 :
use difference question php know
Topic : 6 :
table studio visual eclipse html
Topic : 7 :
way number want best easy
Topic : 8 :
int code long main lines
Topic : 9 :
file error python install command
Topic : 10 :
im code file script use
Topic : 11 :
language good things programming right
Topic : 12 :
like id im way using
Topic : 13 :
div css height content width
Topic : 14 :
loadmodule cprogram override virtual list
Topic : 15 :
search certain grep contain pattern
Topic : 16 :
git branch commit file files
Topic : 17 :
run image windows command process
Topic : 18 :
class methods java private interface
Topic : 19 :
data user request server database
['000', '10', '100', '1000', '11', '12', '123', '13', '14', '15', '16', '20', '23', '25', '26', '30', '32', '50', 

In [25]:
from joblib import dump, load
dump(lda, 'lda.joblib') 
ldaLoad = load('lda.joblib') 

dump(lda, 'lda.joblib') 
ldaLoad = load('lda.joblib') 

print(ldaLoad)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_jobs=1, n_topics=20, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)


In [26]:
tagText.lda_prepare_tag(data_preprocessed)

(LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
              evaluate_every=-1, learning_decay=0.7,
              learning_method='online', learning_offset=50.0,
              max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
              n_jobs=1, n_topics=20, perp_tol=0.1, random_state=0,
              topic_word_prior=None, total_samples=1000000.0, verbose=0),
                00        000     00000    000000  000000000    000001  \
 Topic0   0.053118   0.052715  0.053150  0.052344   0.052576  0.052718   
 Topic1   0.052437   0.052260  0.052836  0.052800   0.052175  0.052540   
 Topic2   0.052550   0.052687  0.052190  0.052302   0.052613  0.051992   
 Topic3   0.052619   0.052946  0.052853  0.052461   0.052848  0.052230   
 Topic4   0.053190   0.052314  0.052989  0.053377   0.052600  0.052558   
 Topic5   0.052709   0.052799  0.053297  0.053076   0.052474  0.053219   
 Topic6   0.052599   0.052951  0.052922  0.052132   0.053396  0.052527   
 Topic7   0.0

In [27]:
lda_tagText, df_topic_keywords, tf_vectorizer = tagText.lda_prepare_tag(data_preprocessed)
            
tagText.lda_predict('git est un outils de code source.', lda_tagText, df_topic_keywords, tf_vectorizer, 5)

'git file files branch want'

In [28]:
from joblib import dump, load
joblib.dump(lda, 'lda.joblib',0)
joblib.dump(df_topic_keywords, 'df_lda_topic_keywords.joblib',0)
joblib.dump(tf_vectorizer, 'tf_lda_vectorizer.joblib',0)

NameError: name 'joblib' is not defined

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
IV.2-b - Data - Tag generator - Unsupervised - MNF</p>

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
no_features=0
X = data_preprocessed[0:9].unique()

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95,
    min_df=2,
    max_features=None,
    stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(X)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

no_topics = 5

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)

Topic : 0 :
git commit files undo pushed local deleted pull question new
Topic : 1 :
int code include main start id new similar piece commit
Topic : 2 :
branch remote git local want pull deleted new pushed include
Topic : 3 :
id question returned similar start want int code commit deleted
Topic : 4 :
result piece returned code want local commit deleted files git


In [30]:
nmf_tagText, tfidf_topic_keywords_tagText, tf_vectorizer = tagText.nmf_prepare_tag(data_preprocessed)
            
tagText.nmf_predict('git est un outils de code source.', nmf_tagText, tfidf_topic_keywords_tagText, tf_vectorizer, 5)

'git branch commit master remote'

In [31]:
from joblib import dump, load
dump(nmf_tagText, 'nmf.joblib') 
nmfLoad = load('nmf.joblib') 
print(nmfLoad)

NMF(alpha=0.1, beta=1, eta=0.1, init='nndsvd', l1_ratio=0.5, max_iter=200,
  n_components=5, nls_max_iter=2000, random_state=1, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)


In [32]:
lda_tagText, df_lda_topic_keywords, tf_lda_vectorizer = tagText.lda_prepare_tag_and_save(data_preprocessed)
mnf_tagText, df_nmf_topic_keywords, tf_mnf_vectorizer = tagText.nmf_prepare_tag_and_save(data_preprocessed)

tagText.lda_predict('git est un outils de code source. C\'est bien', lda_tagText, df_lda_topic_keywords, tf_lda_vectorizer, 5)
tagText.nmf_predict('git est un outils de code source. C\'est bien', mnf_tagText, df_nmf_topic_keywords, tf_mnf_vectorizer, 5)


'git branch commit master remote'

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
IV.3 - Data - Tag generator - Supervised</p>

In [34]:
def superviedTagPurposerFromMessage(message, reload, saveModel):
    tagText = TagText()
    tags = ''
    if reload.lower() == 'true':
        tagText.fileName = 'QuestionVsTags.csv'
        data_question = tagText.read_source()
        if saveModel.lower() == 'true':
            data_preprocessed = data_question['body'].apply(tagText.preprocessing)
            classifier, tags = tagText.supervised_prepare_tag_and_save(data_preprocessed, data_question['SelectedTags'])
            tags = tagText.supervised_predict(message, classifier, tags)
        else:
            data_preprocessed = data_question['body'].apply(tagText.preprocessing)
            classifier, tags = tagText.supervised_prepare_tag(data_preprocessed, data_question['SelectedTags'])
            tags = tagText.supervised_predict(message, classifier, tags)
    else:
        classifier, tags = tagText.supervised_prepare_tag_load()
        tags = tagText.supervised_predict(message, classifier, tags)
    return str(tags)

superviedTagPurposerFromMessage('git est un outils de code source. C\'est bien', 'true', 'False')

[['java', 'c++', 'performance', 'optimization'], ['git', 'version-control', 'git-commit'], ['git', 'git-branch'], ['git'], ['json', 'http-headers', 'content-type'], ['python', 'iterator', 'generator', 'yield', 'coroutine'], ['c++', 'operators', 'code-formatting'], ['git', 'version-control', 'git-commit'], ['git', 'git-branch'], ['javascript', 'jquery', 'redirect'], ['memory-management', 'stack', 'language-agnostic', 'heap', 'dynamic-memory-allocation'], ['git', 'git-commit'], ['javascript', 'function', 'variables', 'scope', 'closures'], ['javascript', 'string', 'substring', 'string-matching'], ['javascript', 'jquery', 'dom', 'visibility'], ['javascript', 'arrays'], ['javascript', 'syntax'], ['html', 'css', 'background-color'], ['git'], ['json', 'comments'], ['javascript', 'function', 'syntax'], ['java', 'date', 'timezone'], ['git', 'branch', 'git-branch'], ['git', 'version-control', 'overwrite'], ['git'], ['c#', '.net', 'string', 'types', 'alias'], ['javascript', 'operators', 'equality

['.htaccess' '.net' '.net-3.5' ..., 'zip' 'zlib' 'zsh']
['c#', 'git', 'java', 'javascript', 'python']


"['c#', 'git', 'java', 'javascript', 'python']"

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
V - Resultat</p>

In [35]:

import numpy as np
import nltk
import pandas as pd
import os
import sys

sys.path.insert(0, './Python/')

from tagText import TagText


tagText = TagText()
print(tagText.urlDirectory)
print(tagText.fileName)


def supervisedTagPurposer():
    return tagPurposer('supervised', 'This is a test to have tag GIT & branche', 'false', 'false')

def supervisedTagPurposerAndReload():
    return tagPurposer('supervised', 'This is a test to have tag GIT & branche', 'true', 'false')

def supervisedTagPurposerAndReloadAndSave():
    return tagPurposer('supervised', 'This is a test to have tag GIT & branche', 'true', 'true')

def tagLDAPurposer():
    return tagPurposer('LDA', 'This is a test to have tag GIT & branche', 'false', 'false')

def tagLDAPurposerAndReload():
    return tagPurposer('LDA', 'This is a test to have tag GIT & branche', 'true', 'false')

def tagLDAPurposerAndReloadAndSave():
    return tagPurposer('LDA', 'This is a test to have tag GIT & branche', 'true', 'true')

def tagNMFPurposer():
    return tagPurposer('NMF', 'This is a test to have tag GIT & branche', 'false', 'false')

def tagNMFPurposerAndReload():
    return tagPurposer('NMF', 'This is a test to have tag GIT & branche', 'true', 'false')

def tagNMFPurposerAndReloadAndSave():
    return tagPurposer('NMF', 'This is a test to have tag GIT & branche', 'true', 'true')

def tagPurposer(mode, message, reload, saveModel):
    if mode.lower() == 'supervised':
        return superviedTagPurposerFromMessage(message, reload.lower(), saveModel.lower())
    if mode.lower() == 'lda' or mode.lower() =='nmf':
        return unsuperviedTagTFIDFPurposerFromMessage(mode.lower(), message, reload.lower(), saveModel.lower())
    return '/i\\Mode indefined/i\\'

def superviedTagPurposerFromMessage(message, reload, saveModel):
    tagText = TagText()
    tags = ''
    if reload.lower() == 'true':
        data_question = tagText.read_source()
        data_preprocessed = data_question['body'].apply(tagText.preprocessing)
        logistic_regression = tagText.supervised_prepare_tag(data_preprocessed)
        tags = tagText.supervised_predict(message, logistic_regression, 5)
    else:
        logistic_regression = tagText.supervised_prepare_tag_load()
        tags = tagText.supervised_predict(message, logistic_regression, 5)
    return tags

def unsuperviedTagTFIDFPurposerFromMessage(option, message, reloadModel, saveModel):
    tagText = TagText()
    tags = ''
    if option.lower() != 'lda' and option.lower() != 'nmf':
        return '/i\\Mode indefined/i\\'
    if reloadModel.lower() == 'true':
        data_question = tagText.read_source()
        data_preprocessed = data_question['body'].apply(tagText.preprocessing)
        if option.lower() == 'lda':
            if saveModel.lower() == 'true':
                lda_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText = tagText.lda_prepare_tag_and_save(data_preprocessed)
                tags = tagText.lda_predict(message, lda_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText, 5)
            else:
                lda_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText = tagText.lda_prepare_tag(data_preprocessed)
                tags = tagText.lda_predict(message, lda_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText, 5)
        if option.lower() == 'nmf':
            if saveModel.lower() == 'true':
                nmf_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText = tagText.nmf_prepare_tag_and_save(data_preprocessed)
                tags = tagText.nmf_predict(message, nmf_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText, 5)
            else:
                nmf_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText = tagText.nmf_prepare_tag(data_preprocessed)
                tags = tagText.nmf_predict(message, nmf_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText, 5)
    else:
        if option.lower() == 'lda':
            lda_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText = tagText.lda_prepare_tag_load()
            tags = tagText.lda_predict(message, lda_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText, 5)
        if option.lower() == 'nmf':
            nmf_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText = tagText.nmf_prepare_tag_load()
            tags = tagText.nmf_predict(message, nmf_tagText, df_topic_keywords_tagText, tf_vectorizer_tagText, 5)
    return tags


Data/
QuestionVsTags.csv


In [36]:
tagLDAPurposerAndReloadAndSave()

'git file files branch want'

Une API :  https://ejacques.pythonanywhere.com/apiTagGenerator/

Un depot GIT : https://github.com/EricJacquesPro/TextCategorization/